# Read examples

In [ ]:
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.interfaces.elegant import elegant_h5_to_data

# elegant hdf5 format

This will convert to a data dict

In [ ]:
data = elegant_h5_to_data("data/elegant_raw.h5")
data

Create `ParticleGroup` and plot:

In [ ]:
P = ParticleGroup(data=data)
P.plot("delta_t", "delta_pz")

# Genesis4 HDF5 format

Genesis4 stores particles in `z` slices in its HDF5 file format. Each slice has a `z` length equal to the reference wavelength `lambda0`. In order to speed up the simulation, Genesis4 also has an integer parameter [`sample`](https://github.com/svenreiche/Genesis-1.3-Version4/blob/master/manual/MAIN_INPUT.md#time) that essentially skips adjacent slices, defined in the input file:


In [ ]:
# Input
!cat data/genesis4/genesis4.in

In [ ]:
# Lattice
!cat data/genesis4/genesis4.lat

Running Genesis4 on this input (`genesis4 genesis4.in`) produces the raw particle file `end.par.h5` with slice particle.

In [ ]:
parfile = "data/genesis4/end.par.h5"

By default `ParticleGroup.from_genesis4` will load these particles relative to their slice position. Because `sample=10` we see a comb-like distribution:

In [ ]:
P_default = ParticleGroup.from_genesis4(parfile)
P_default.plot("z", "pz", bins=200)
P_default

In [ ]:
# ParticleGroup has a `smear` option that shifts particles in each slice by a random integer multiple of the slice spacing, so that the overall distrubution is smoother while preserving the bunching characteristics:

In [ ]:
P_smear = ParticleGroup.from_genesis4(parfile, smear=True)
P_smear.plot("z", "pz", bins=100)
P_smear

Looking closer:

In [ ]:
P = P_default
P[(P.z > 30e-6) & (P.z < 35e-6)].plot("z", "pz", bins=100)
P

In [ ]:
P = P_smear
P[(P.z > 30e-6) & (P.z < 35e-6)].plot("z", "pz", bins=100)
P

Normally Genesis4 populates the same number of particles in each slice, with each slice having its own weight (internally the `current`). In some cases it's simpler to have particles that have the same weight, so `ParticleGroup` further has an `equal_weights` option that samples particles from the slices according to the relative weights. Notice that there are fewer particles in this case, but that the projected densities are roughtly the same as above.

In [ ]:
P_equal = ParticleGroup.from_genesis4(parfile, smear=True, equal_weights=True)
P_equal.plot("z", "pz", bins=100)
P_equal